## 1. Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Mecab
from gensim import corpora, models
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.filterwarnings('ignore')
from gensim.models.coherencemodel import CoherenceModel
from wordcloud import WordCloud,ImageColorGenerator
from gensim.models.ldamodel import LdaModel
from gensim.test.utils import datapath
from gensim.corpora.dictionary import Dictionary
import matplotlib
from matplotlib import font_manager, rc
import platform
import time

In [ ]:
#한글 글꼴 깨짐 방지
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

## 2. 데이터 불러오기

In [ ]:
data = pd.read_excel('최종데이터.xlsx')

## 3. nouns 공백으로 분리

In [ ]:
data['nouns'] = [text.split(" ") for text in data['nouns']]

## 4. 연도별로 분리 

In [ ]:
data21 = data[data.Year == 2021]
data20 = data[data.Year == 2020]
data19 = data[data.Year == 2019]
data18 = data[data.Year == 2018]
data17 = data[data.Year == 2017]
data16 = data[data.Year == 2016]

## 5. LDA

### 5-1. Parameter Tuning

In [ ]:
##연도별 토픽 수(K) 최적화
datas = [data16,data17,data18,data19,data20,data21]
year = ['2016','2017','2018','2019','2020','2021']

for i,data in enumerate(datas):
    
    df_noun = data.nouns
    dictionary = corpora.Dictionary(df_noun)
    corpus = [dictionary.doc2bow(text) for text in df_noun]
    coherences = []
    perplexities = []

    num_topic_list = [5,6,7,8,9,10]
    # 각각의 토픽 수로 LDA 모델 학습
    for nt in num_topic_list:
    
            
        
        tic = time.time()
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = nt, id2word=dictionary, passes = 10,random_state = 0)
        print('epoch :',10, 'ntopic : ',nt,'걸린시간 : ', time.time()-tic)

        topics = ldamodel.print_topics(num_words = 8)
        for topic in topics:
            print(topic)

        cm = CoherenceModel(model = ldamodel, corpus = corpus, coherence = 'u_mass') #Coherence Score
        coherence = cm.get_coherence()
        print('Coherence :', coherence)
        coherences.append(coherence)
        perplexities.append(ldamodel.log_perplexity(corpus)) #Perplexity Score
            
        print('Perplexity :', ldamodel.log_perplexity(corpus),'\n')
        print('---------------------------------------------')
        
    # 데이터 토픽 수에 따른 Coherence Score 시각화
    x = num_topic_list
    plt.figure(figsize=(10,10))
    plt.subplot(211)
    plt.plot(x,coherences)
    plt.xlabel('num_topic')
    plt.ylabel('coherence score')
    plt.title(year[i]+'년 데이터 토픽 수에 따른 Coherence')
    
    #데이터 토픽수에 따른 Perplexity Score 시각화
    plt.subplot(212)
    plt.plot(x,perplexities)
    plt.xlabel('num_topic')
    plt.ylabel('perplexity score')
    plt.title(year[i]+'년 데이터 토픽 수에 따른 Perplexity')
    
    plt.show()
    print('\n--------------------------------------------------------------------------------------------------------------------')

In [ ]:
#a 파라미터 설정
datas = [data16,data17,data18,data19,data20,data21]
num_topic_list = [5,6,9,6,7,6]
year = ['2016','2017','2018','2019','2020','2021']

# 데이터와 최적화한 K값 매칭
for i,(data, nt) in enumerate(zip(datas,num_topic_list)):
    
    df_noun = data.nouns
    dictionary = corpora.Dictionary(df_noun)
    corpus = [dictionary.doc2bow(text) for text in df_noun]
    coherences = []
    perplexities = []

    
    alphas = ['symmetric','auto',nt/50, 0.1] #알파값 후보로 4가지 설정

#  alpha값 최적화
    for alpha in alphas:
            
        
        tic = time.time()
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = nt, id2word=dictionary, passes = 15, alpha = alpha, random_state = 0)
        print('epoch :',15,'alpha :',alpha , 'ntopic : ',nt,'걸린시간 : ', time.time()-tic)

        topics = ldamodel.print_topics(num_words = 7)
        for topic in topics:
            print(topic)

        cm = CoherenceModel(model = ldamodel, corpus = corpus, coherence = 'u_mass')
        coherence = cm.get_coherence()
        print('Coherence :', coherence)
        coherences.append(coherence)
        perplexities.append(ldamodel.log_perplexity(corpus))
            
        print('Perplexity :', ldamodel.log_perplexity(corpus),'\n')
        print('---------------------------------------------')
    
    #알파값에 따른 Coherence, Perplexity 시각화
    
    x = alphas
    plt.figure(figsize=(10,10))
    plt.subplot(211)
    plt.plot(x,coherences)
    plt.xlabel('alpha')
    plt.ylabel('coherence score')
    plt.title(year[i]+'년 데이터 alpha 에 따른 Coherence')

    plt.subplot(212)
    plt.plot(x,perplexities)
    plt.xlabel('alpha')
    plt.ylabel('perplexity score')
    plt.title(year[i]+'년 데이터 alpha 에 따른 Perplexity')
    
    plt.show()
    print('\n--------------------------------------------------------------------------------------------------------------------')


### 5-2. 2016년 LDA

- 다른 년도의 프로세스도 동일하게 진행

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data16.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data16.nouns]
print(corpus[1])

In [ ]:
# lda 모델 학습(파라미터 지정)
ldamodel16 = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes = 15, alpha = 0.1,eta = 0.01, per_word_topics = True)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel16, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel16.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel16.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel16,corpus, dictionary)
vis

### 5-3. 2017

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data17.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data17.nouns]
print(corpus[1])

In [ ]:
# lda 학습
ldamodel17 = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes = 15, alpha = 0.1,eta = 0.01, per_word_topics = True)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel17, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel17.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel17.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel17,corpus, dictionary)
vis

### 5-4. 2018

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data18.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data18.nouns]
print(corpus[1])

In [ ]:
# lda 모델 학습(파라미터 지정)
ldamodel18 = gensim.models.ldamodel.LdaModel(corpus,num_topics = 6, id2word = dictionary, passes = 50, random_state = 0, alpha = 0.05,eval_every = 5,
                                             eta = 0.001)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel18, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel18.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel18.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel18,corpus, dictionary)
vis

### 5-5. 2019

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data19.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data19.nouns]
print(corpus[1])

In [ ]:
# lda 학습
ldamodel19 = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes = 15, alpha = 0.1,eta = 0.01, per_word_topics = True)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel19, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel19.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel19.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel19,corpus, dictionary)
vis

### 5-6. 2020

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data20.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data20.nouns]
print(corpus[1])

In [ ]:
# lda 모델 학습(파라미터 지정)
ldamodel20 = gensim.models.ldamodel.LdaModel(corpus,num_topics = 6, id2word = dictionary, passes = 50, random_state = 0, alpha = 0.05,eval_every = 5,
                                             eta = 0.001)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel20, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel20.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel20.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel20,corpus, dictionary)
vis

### 5-7. 2021

In [ ]:
# 각 문서별 단어의 빈도수 계산
dictionary = corpora.Dictionary(data21.nouns)
# (단어의 정수인코딩 값, 단어 출현 빈도수)의 형태로 변환
corpus = [dictionary.doc2bow(text) for text in data21.nouns]
print(corpus[1])

In [ ]:
# lda 학습
ldamodel21 = gensim.models.ldamodel.LdaModel(corpus, num_topics = 6, id2word=dictionary, passes = 15, alpha = 0.1,eta = 0.01, per_word_topics = True)

# 평가지표 점수 출력
cm = CoherenceModel(model = ldamodel21, corpus = corpus, coherence = 'u_mass')
coherence = cm.get_coherence()
print('Coherence :', coherence)
print('Perplexity :', ldamodel21.log_perplexity(corpus))

In [ ]:
# 토픽별 키워드 8개씩 추출
ldamodel21.print_topics(num_words = 8)

In [ ]:
# pyLDAvis를 사용하여 LDA 시각화
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel21,corpus, dictionary)
vis

## 6. WordCloud

### 6-1. 2016

In [ ]:
# 토픽명 리스트
topic_list = ['친환경 에너지','하이브리드','테슬라 모델X','전기차 주행','르노 삼성 전기차 관련','아이오닉 하이브리드']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2016 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel16.print_topics())):
    
    if t == 1 :
        pass
    
    if t == 6 :
        plt.subplot(5,3,2)
        x = dict(ldamodel16.show_topic(t,100))
        im = wc.generate_from_frequencies(x)
        plt.imshow(im)
        plt.axis("off")
        plt.title(topic_list[1], fontsize = 18)
    else :
        plt.subplot(5,3,t+1)
        x = dict(ldamodel16.show_topic(t,100))
        im = wc.generate_from_frequencies(x)
        plt.imshow(im)
        plt.axis("off")
        plt.title("Topic #" + str(t))
        plt.title(topic_list[t], fontsize = 18)
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0.04, wspace = 0.04)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2016.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

### 6-2. 2017

In [ ]:
# 토픽명 리스트
topic_list = ['전기차 렌트','전기차 SUV','하이브리드와 전기차 비교','전기차 구매','전기차 충전','테슬라 모델3']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2017 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel17.print_topics())):
    
    if t == 3 :
        pass 
    elif t == 6 :
        plt.subplot(5,3,4)
        x = dict(ldamodel17.show_topic(t,100))
        im = wc.generate_from_frequencies(x)
        plt.imshow(im)
        plt.axis("off")
        plt.title(topic_list[3], fontsize = 18)
    
    
    else :
        plt.subplot(5,3,t+1)
        x = dict(ldamodel17.show_topic(t,100))
        im = wc.generate_from_frequencies(x)
        plt.imshow(im)
        plt.axis("off")
        plt.title(topic_list[t], fontsize = 18)
        plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0.04, wspace = 0.04)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2017.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

### 6-3. 2018

In [ ]:
# 토픽명 리스트
topic_list = ['전기차 구매','전기차 키워드', '전기차 주행','테슬라','전기차 충전','전기차 렌트']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2018 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel18.print_topics())):
    plt.subplot(5,3,t+1)
    x = dict(ldamodel18.show_topic(t,100))
    im = wc.generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title(topic_list[t], fontsize = 18)
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0.04, wspace = 0.04)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2018.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

### 6-4. 2019

In [ ]:
# 토픽명 리스트
topic_list = ['전기차 주행','전기차 키워드', '전기차 구매','전기차 충전','전기차 렌트','테슬라']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2019 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel19.print_topics())):
    plt.subplot(5,3,t+1)
    x = dict(ldamodel19.show_topic(t,100))
    im = wc.generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title(topic_list[t], fontsize = 18)
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0.04, wspace = 0.04)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2019.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

### 6-5. 2020

In [ ]:
# 토픽명 리스트
topic_list = ['차종 비교','배터리, 충전 이슈','전기차 구매','현대와 테슬라 비교','전기차 키워드','전기차 충전']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2020 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel20.print_topics())):
    plt.subplot(5,3,t+1)
    x = dict(ldamodel20.show_topic(t,100))
    im = wc.generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title(topic_list[t], fontsize = 18)
    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0.04, wspace = 0.04)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2020.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

### 6-6. 2021

In [ ]:
# 토픽명 리스트
topic_list = ['배터리, 충전 이슈','전기차 충전','자율 주행','전기차 구매','현대와 테슬라 비교','차종 비교']

# 워드클라우드 설정(폰트와 배경)
wc = WordCloud(background_color='white',font_path='C:/Users/admin/AppData/Local/Microsoft/Windows/Fonts/서울남산 장체B.ttf')            # 2021 워드클라우드

# 워드클라우드 그리기(6개의 토픽)
plt.figure(figsize=(20,20))
for t in range(len(ldamodel21.print_topics())):
    plt.subplot(5,3,t+1)
    x = dict(ldamodel21.show_topic(t,100))
    im = wc.generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title(topic_list[t], fontsize = 18)

    plt.tight_layout()
    plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0.01)

# 출력된 이미지 저장    
plt.savefig('./LDA/Wordcloud/wc2021.png',bbox_inces='tight', 
            pad_inches=0, 
            dpi=100)

## 7. LDA Model Save & Load

In [ ]:
# 학습한 Model을 disk에 저장
ldamodel16.save('lda_2016.lda')
ldamodel17.save('lda_2017.lda')
ldamodel18.save('lda_2018.lda')
ldamodel19.save('lda_2019.lda')
ldamodel20.save('lda_2020.lda')
ldamodel21.save('lda_2021.lda')

In [ ]:
# 저장한 Model 불러오기
ldamodel16 = LdaModel.load('lda_2016.lda')
ldamodel17 = LdaModel.load('lda_2017.lda')
ldamodel18 = LdaModel.load('lda_2018.lda')
ldamodel19 = LdaModel.load('lda_2019.lda')
ldamodel21 = LdaModel.load('lda_2020.lda')
ldamodel21 = LdaModel.load('lda_2021.lda')